In [ ]:
# SIMPLE NLP ANALYSIS USING HUGGINGFACE

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [16]:
url = 'https://github.com/dasarpai/DAI-Datasets/blob/main/zomato_reviews.csv'
url = "https://raw.githubusercontent.com/dasarpai/DAI-Datasets/main/zomato_reviews.csv"

# Read the CSV file directly from GitHub
df = pd.read_csv(url, index_col="Unnamed: 0")

In [17]:
df.head()

,rating,review
0,5,nice
1,5,"best biryani , so supportive staff of outlet ,..."
2,4,delivery boy was very decent and supportive.👌👍
3,1,"worst biryani i have tasted in my life, half o..."
4,5,all food is good and tasty . will order again ...


In [2]:
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_03:03:05_Pacific_Daylight_Time_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [3]:
!nvidia-smi


Fri Nov  8 21:13:44 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.81                 Driver Version: 560.81         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4070 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   53C    P8              8W /   40W |       0MiB /   8188MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
import torch
print(torch.cuda.is_available())  # Should print True if GPU is available
print(torch.cuda.device_count())  # Should print number of GPUs available

False
0


In [2]:
from transformers import pipeline
classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)
example_results = classifier("I love this!")

c:\Python312\Lib\site-packages\transformers\pipelines\text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [ ]:
# classifier = classifier.model.to('cuda')

In [19]:
example_results

[[{'label': 'anger', 'score': 0.004419790115207434},
  {'label': 'disgust', 'score': 0.001611992483958602},
  {'label': 'fear', 'score': 0.0004138525982853025},
  {'label': 'joy', 'score': 0.9771687984466553},
  {'label': 'neutral', 'score': 0.005764586851000786},
  {'label': 'sadness', 'score': 0.002092392183840275},
  {'label': 'surprise', 'score': 0.008528688922524452}]]

In [40]:
example_best = 0
example_best_score = 0 

# Access the first element (the list) in example_results
for idx, e in enumerate(example_results[0]):  # Access the first element of the outer list
    if e['score'] > example_best_score:
        example_best_score = e['score']
        example_best = idx

# Output the best result (the dictionary with the highest score)
print(example_results[0][example_best])


{'label': 'joy', 'score': 0.9771687984466553}


In [42]:
reviews = df['review']

In [43]:
df['review'] = df['review'].fillna('Null filled with this placeholder message.')

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5479 entries, 0 to 5478
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   rating  5479 non-null   int64 
 1   review  5479 non-null   object
dtypes: int64(1), object(1)
memory usage: 128.4+ KB


In [47]:
best_score = 0
best_idx = 0
for idx, r in enumerate(result[0]):
    if isinstance(r, dict) and 'score' in r:  # if r is a dictionary and has the 'score' key
        if r['score'] > best_score:
            best_score = r['score']
            best_idx = idx
    elif isinstance(r, list) or isinstance(r, tuple):  # if r is a list/tuple
        # Modify the index based on where the 'score' is in your list/tuple
        if r[1] > best_score:  # assuming score is at index 1
            best_score = r[1]
            best_idx = idx

print(result[0][best_idx])  # Print the best result


{'label': 'neutral', 'score': 0.7410889863967896}


In [51]:
review_sentiments = []
for rev in reviews:
    best_idx = 0
    best_score = 0
#     print(rev)
    result = classifier(rev)
    for idx,r in enumerate(result[0]):
#         print(r)
        if r['score']>best_score:
            best_score = r['score']
            best_idx = idx
            
    review_sentiments.append(result[0][best_idx])

In [53]:
r

{'label': 'surprise', 'score': 0.014902020804584026}

In [54]:
review_sentiments

[{'label': 'neutral', 'score': 0.7410889863967896},
 {'label': 'neutral', 'score': 0.2890426516532898},
 {'label': 'joy', 'score': 0.5907884240150452},
 {'label': 'disgust', 'score': 0.9389179944992065},
 {'label': 'joy', 'score': 0.7750592231750488},
 {'label': 'joy', 'score': 0.5231075882911682},
 {'label': 'neutral', 'score': 0.7880570292472839},
 {'label': 'joy', 'score': 0.4599943161010742},
 {'label': 'neutral', 'score': 0.5484577417373657},
 {'label': 'joy', 'score': 0.8362355828285217},
 {'label': 'neutral', 'score': 0.3215256333351135},
 {'label': 'joy', 'score': 0.985458493232727},
 {'label': 'joy', 'score': 0.9684831500053406},
 {'label': 'joy', 'score': 0.7866860628128052},
 {'label': 'joy', 'score': 0.9407974481582642},
 {'label': 'joy', 'score': 0.6331069469451904},
 {'label': 'sadness', 'score': 0.949017345905304},
 {'label': 'sadness', 'score': 0.7905277609825134},
 {'label': 'joy', 'score': 0.6387940049171448},
 {'label': 'joy', 'score': 0.772070050239563},
 {'label': 

In [55]:
sentiments = []
scores = []

for rev_sent in review_sentiments:
    sentiments.append(rev_sent['label'])
    scores.append(rev_sent['score'])    

In [56]:
df['sentiment'] = sentiments
df['scores'] = scores

In [57]:
df.head()

,rating,review,sentiment,scores
0,5,nice,neutral,0.741089
1,5,"best biryani , so supportive staff of outlet ,...",neutral,0.289043
2,4,delivery boy was very decent and supportive.👌👍,joy,0.590788
3,1,"worst biryani i have tasted in my life, half o...",disgust,0.938918
4,5,all food is good and tasty . will order again ...,joy,0.775059


In [58]:
# the count of rows where we have a rating of 5 but our sentiment analysis isn't joy
df[(df['rating']==5) & (df['sentiment']!='joy')]['rating'].count()

1776

In [59]:
# the count of rows where we have a rating of 5 and our sentiment analysis is joy
df[(df['rating']==5) & (df['sentiment']=='joy')]['rating'].count()

512

In [60]:
df[df['rating']==5]['rating'].count()

2288

In [61]:
# as we can see above only 512 of our 1776 results are marked as joy which is quite low
df[(df['rating']==5) & (df['sentiment']!='joy')].head(20)

,rating,review,sentiment,scores
0,5,nice,neutral,0.741089
1,5,"best biryani , so supportive staff of outlet ,...",neutral,0.289043
6,5,overall good experience . will order again,neutral,0.788057
8,5,tasty food . homestyle tasty prepration.,neutral,0.548458
20,5,"tasteis best , zabardast mouthwatering . must ...",fear,0.371340
23,5,bawarchi satisfied there customer needs🤩,neutral,0.482653
24,5,Too Helping nature of resturent owners,neutral,0.414461
25,5,"i have tried many items from bawarchi , these ...",sadness,0.234048
29,5,Free pizza was not given to me. Pizza hut does...,neutral,0.482814
35,5,Very bad. Waste of money Pizza Hut no more lik...,disgust,0.800306


In [62]:
# the reason for the problematic findings like that is the randomly rated columns
# let's check these 2 rows

df.iloc[[35,36]]

,rating,review,sentiment,scores
35,5,Very bad. Waste of money Pizza Hut no more lik...,disgust,0.800306
36,5,pizza got cold........waste of money,anger,0.461766


In [ ]:
# these 2 rows above obviously are negative results and sentiment analysis results 
# are also showing that finding


In [ ]:
# THE TAKEAWAY IS WE CAN NOT TRUST MERE RATINGS FROM THE USERS AND WE ALSO NEED TO CHECK 
# WHAT THEY REALLY MEANT IF THEY PROVIDED A COMMENT :D